---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 1 - Creating and Manipulating Graphs

Eight employees at a small company were asked to choose 3 movies that they would most enjoy watching for the upcoming company movie night. These choices are stored in the file `Employee_Movie_Choices.txt`.

A second file, `Employee_Relationships.txt`, has data on the relationships between different coworkers. 

The relationship score has value of `-100` (Enemies) to `+100` (Best Friends). A value of zero means the two employees haven't interacted or are indifferent.

Both files are tab delimited.

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
from networkx.algorithms import bipartite


# This is the set of employees
employees = set(['Pablo',
                 'Lee',
                 'Georgia',
                 'Vincent',
                 'Andy',
                 'Frida',
                 'Joan',
                 'Claude'])

# This is the set of movies
movies = set(['The Shawshank Redemption',
              'Forrest Gump',
              'The Matrix',
              'Anaconda',
              'The Social Network',
              'The Godfather',
              'Monty Python and the Holy Grail',
              'Snakes on a Plane',
              'Kung Fu Panda',
              'The Dark Knight',
              'Mean Girls'])


# you can use the following function to plot graphs
# make sure to comment it out before submitting to the autograder
def plot_graph(G, weight_name=None):
    '''
    G: a networkx G
    weight_name: name of the attribute for plotting edge weights (if G is weighted)
    '''

    import matplotlib.pyplot as plt
    
    plt.figure()
    pos = nx.spring_layout(G)
    edges = G.edges()
    weights = None
    
    if weight_name:
        weights = [int(G[u][v][weight_name]) for u,v in edges]
        labels = nx.get_edge_attributes(G,weight_name)
        nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)
        nx.draw_networkx(G, pos, edges=edges, width=weights);
    else:
        nx.draw_networkx(G, pos, edges=edges);

### Question 1

Using NetworkX, load in the bipartite graph from `Employee_Movie_Choices.txt` and return that graph.

*This function should return a networkx graph with 19 nodes and 24 edges*

In [72]:
def answer_one():
         
    G = nx.Graph()
    movie_prefs = pd.read_csv("Employee_Movie_Choices.txt", delimiter = '\t')
        
    Employee_set = set(movie_prefs['#Employee'])
    Movie_set = set(movie_prefs['Movie'])
    
    
    G.add_nodes_from(Employee_set, bipartite=0)
    G.add_nodes_from(Movie_set, bipartite=1)
    
    Employee_Movie_edges = list(zip(movie_prefs['#Employee'],movie_prefs['Movie']))
    G.add_edges_from(Employee_Movie_edges)
    
       
    return G

### Question 2

Using the graph from the previous question, add nodes attributes named `'type'` where movies have the value `'movie'` and employees have the value `'employee'` and return that graph.

*This function should return a networkx graph with node attributes `{'type': 'movie'}` or `{'type': 'employee'}`*

In [71]:
def answer_two():
    
    G = nx.Graph()
    movie_prefs = pd.read_csv("Employee_Movie_Choices.txt", delimiter = '\t')
        
    Employee_set = set(movie_prefs['#Employee'])
    Movie_set = set(movie_prefs['Movie'])
    
    G.add_nodes_from(Employee_set, bipartite=0, type = 'employee')
    G.add_nodes_from(Movie_set, bipartite=1, type = 'movie')
    
    
    Employee_Movie_edges = list(zip(movie_prefs['#Employee'],movie_prefs['Movie']))
    G.add_edges_from(Employee_Movie_edges)
         
    return G

### Question 3

Find a weighted projection of the graph from `answer_two` which tells us how many movies different pairs of employees have in common.

*This function should return a weighted projected graph.*

In [70]:
from networkx.algorithms import bipartite

def answer_three():
        
    G = nx.Graph()
    movie_prefs = pd.read_csv("Employee_Movie_Choices.txt", delimiter = '\t')
        
    Employee_set = set(movie_prefs['#Employee'])
    Movie_set = set(movie_prefs['Movie'])
    
    G.add_nodes_from(Employee_set, bipartite=0, type = 'employee')
    G.add_nodes_from(Movie_set, bipartite=1, type = 'movie')
    
    
    Employee_Movie_edges = list(zip(movie_prefs['#Employee'],movie_prefs['Movie']))    
    G.add_edges_from(Employee_Movie_edges)
    
    adj_list = []
    for line in nx.generate_adjlist(G):
        adj_list.append(line)
    
    P = bipartite.weighted_projected_graph(G,Employee_set)
    
    return P

### Question 4

Suppose you'd like to find out if people that have a high relationship score also like the same types of movies.

Find the Pearson correlation ( using `DataFrame.corr()` ) between employee relationship scores and the number of movies they have in common. If two employees have no movies in common it should be treated as a 0, not a missing value, and should be included in the correlation calculation.

*This function should return a float.*

In [69]:
def answer_four():
    
    employee_relationships = pd.read_csv('Employee_Relationships.txt', delimiter = '\t', header=None)
    employee_movie_choices = pd.read_csv('Employee_Movie_Choices.txt', delimiter = '\t')
    
    employee_set = set(employee_movie_choices['#Employee'])
    movie_set = set(employee_movie_choices['Movie'])
     
    G = nx.Graph()
    G.add_nodes_from(employee_set, bipartite=0)
    G.add_nodes_from(movie_set, bipartite=1)
    employee_movie_edges = list(zip(employee_movie_choices['#Employee'],employee_movie_choices['Movie']))
    G.add_edges_from(employee_movie_edges)
    P = bipartite.weighted_projected_graph(G,employee_set)
    
    G1 = nx.from_pandas_dataframe(employee_relationships,0,1,edge_attr =2)
    G1.add_edges_from(P.edges(data=True))
        
    df = pd.DataFrame(G1.edges(data=True))
    column_to_delineate = list(df[2])
    relationship_score_column = []
    movies_in_common_column = []
    for i,row in enumerate(column_to_delineate):
        relationship_score_column.append(row[2])
        try:
            movies_in_common_column.append(row['weight'])
        except:
            movies_in_common_column.append(0)
    
    df2 = pd.DataFrame()
    df2['relationship_score'] = relationship_score_column
    df2['movies_in_common'] = movies_in_common_column
    
    answer = df2.corr().iloc[0][1]
       
    return answer

0.78839622217334759